### Repeatable, Parallel Model Training and Model Tracking and Analysis  
  
In this notebook, you will use Azure Machine learning to create a repeatable, scaleable prediction framework. Key concepts in this notebook are:  
  
1. **Create and Run Inference Pipeline**
   - The [model inference script](../src/parallel_inference.py) is custom and interchangeable so it can be easily adapted for changing needs
   - The same compute cluster, and thus parallel and scaleable concepts, from [Step 1 - Model Training](./1_training_pipeline.ipynb) are used
   - The inference script uses MLFlow to automatically select and use the "best" model version from a given partitions registered models
     - For this notebook, the lowest test RMSE score is how "best" is defined
  
1. **Azure Machine Learning Endpoint / Deployment**
   -  You will also deploy the prediction pipeline to an Azure Machine Learning Batch Endpoint, this allows for:
      -  Scheduling or a programatic method to run the pipeline to generate predictions
      -  Deployment of multiple pipeline to the same endpoint for ease or conducting side-by-side comparison

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, Input, Output, load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import BatchEndpoint, PipelineComponentBatchDeployment, Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.parallel import parallel_run_function, RunFunction
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import glob
import os
import time

In [ ]:
# Initialize ML Client
load_dotenv(find_dotenv(), override=True)

# authenticate
credential = DefaultAzureCredential(tenantid=os.environ.get('TENANT_ID'))

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

print(dict(os.environ))

### Define some input parameters

In [ ]:
# Define Environment
custom_env_name = "mm-remote-env-py310"

# Define Compute Target
cpu_compute_target = "mm-cpu-cluster"

# Define mlflow tracking URI
azureml_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri

In [ ]:
# Acess data asset
data_name = "oj-sim-sales-test"
test_data_asset = ml_client.data.get(data_name, label='latest')
print(test_data_asset.path)

In [ ]:
# Load data preparation component
partition_data = load_component(source="../src/components/partition_data/partition_data.yaml")

### Define and Run Pipeline

In [ ]:
# Create parallel inference step
# Declare parallel job, with a run_function task
many_model_inference_with_partition_keys = parallel_run_function(
    name="distributed_inference",
    display_name="Many Model Predictions",
    description="parallel job to batch predict with many models",
    inputs=dict(
        data_source=Input(
            type=AssetTypes.MLTABLE,
            description="Input mltable with predefined partition format.",
            mode=InputOutputModes.DIRECT,  # [Important] To use 'partition_keys', input MLTable is required to use 'direct' mode.
        ),
        drop_cols=Input(
            type="string",
            description="Columns need to be dropped before training. Split by comma.",
        ),
        date_col=Input(
            type="string",
            description="Name of date column in data",
        ),
        tracking_uri=Input(
            type="string",
            description="tracking uri of mlflow server (aml workspace)",
        ),
    ),
    outputs=dict(
        output_dir=Output(
            type=AssetTypes.URI_FOLDER,
            mode=InputOutputModes.RW_MOUNT,
        )
    ),
    input_data="${{inputs.data_source}}",  # Define which input data will be splitted into mini-batches
    partition_keys=["Store","Brand"],  # Use 'partition_keys' as the data division method. This method requires MLTable input with partition setting pre-defined in MLTable artifact.
    instance_count=6,  # Use X nodes from compute cluster to run this parallel job.
    max_concurrency_per_instance=2,  # Create 2 worker processors in each compute node to execute mini-batches.
    error_threshold=-1,  # Monitor the failures of item processed by the gap between mini-batch input count and returns. 'Many model training' scenario doesn't fit this setting and '-1' means ignore counting failure items by mini-batch returns.
    mini_batch_error_threshold=5,  # Monitor the failed mini-batch by exception, time out, or null return. When failed mini-batch count is higher than this setting, the parallel job will be marked as 'failed'.
    retry_settings=dict(
        max_retries=1,  # Define how many retries when mini-batch execution is failed by exception, time out, or null return.
        timeout=60,  # Define the timeout in second for each mini-batch execution.
    ),
    logging_level="DEBUG", # DEBUG, INFO, WARNING, ERROR, ETC
    task=RunFunction(
        code="../src/",
        entry_script="parallel_inference.py",
        environment=ml_client.environments.get(custom_env_name, label="latest"),
        program_arguments="--drop_cols ${{inputs.drop_cols}} "  # Passthrough input parameters into parallel_train script.
        "--tracking_uri ${{inputs.tracking_uri}} "
        "--date_col ${{inputs.date_col}} "
        "--output_dir ${{outputs.output_dir}} ",
    ),
)

In [ ]:
# Build Pipeline
# Declare the overall input of the job.
test_oj_data = Input(
    path=test_data_asset.path,
    type=AssetTypes.URI_FILE,
    mode=InputOutputModes.RO_MOUNT,
)

# Declare pipeline structure.
@pipeline(display_name="inference pipeline")
def parallel_inference_pipeline(pipeline_input_data: Input(type=AssetTypes.MLTABLE)):
    
    # Declare 1st data partition command job.
    partition_job = partition_data(
        data_source=pipeline_input_data,
        partition_keys="Store,Brand",
    )

    # Declare 2nd parallel model training job.
    parallel_inference = many_model_inference_with_partition_keys(
        data_source=partition_job.outputs.tabular_output_data,
        drop_cols="Advert,Store,Brand",
        date_col="WeekStarting",
        tracking_uri=azureml_tracking_uri
    )
    
    return {"pipeline_output": parallel_inference.outputs.output_dir}

    # User could override parallel job run-level property when invoke that parallel job/component in pipeline.
    # parallel_train.resources.instance_count = 5
    # parallel_train.max_concurrency_per_instance = 2
    # parallel_train.mini_batch_error_threshold = 10

# Create pipeline instance
inference_pipeline = parallel_inference_pipeline(pipeline_input_data=test_oj_data,)

# Set pipeline level compute
inference_pipeline.settings.default_compute = cpu_compute_target
print(inference_pipeline)

In [ ]:
# Run Inference Pipeline (optional)
pipeline_job = ml_client.jobs.create_or_update(
    inference_pipeline,
    experiment_name="many-models-parallel-inference-job",
)

### Deploy as a Batch Endpoint

In [ ]:
# Create Batch Endpoint

endpoint_name = "manymodels-batch-endp"
print(f"Endpoint name: {endpoint_name}")

endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A many models batch inference endpoint",
)

try:
    ml_client.batch_endpoints.get(endpoint_name)
    print(f"'{endpoint_name}' endpoint already exists. Will re-use existing endpoint")
except Exception as e:
    print("No existing endpoint found. Creating new endpoint....")
    ml_client.batch_endpoints.begin_create_or_update(endpoint).result()
    print("Complete.")

In [ ]:
# Define Pipeline Component
pipeline_component = parallel_inference_pipeline().component

# Register Pipeline Component for better tracking and versioning
ml_client.components.create_or_update(pipeline_component)

In [ ]:
# Create Deployment
deployment_name = "mm-inference-deployment"

deployment = PipelineComponentBatchDeployment(
    name=deployment_name,
    description="A many models deployment.",
    endpoint_name=endpoint_name,
    component=pipeline_component,
    settings={"default_compute": cpu_compute_target},
)

try:
    ml_client.batch_deployments.get(name=deployment_name, endpoint_name=endpoint_name)
    print(f"'{deployment_name}' already exists. Will re-use existing.")
except Exception as e:
    print("No existing deployment found. Creating new deployment....")
    ml_client.batch_deployments.begin_create_or_update(deployment).result()
    print("Complete.")

In [ ]:
# Invoke Batch endpoint
input_data = test_oj_data

endp_job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_name,
    inputs={"pipeline_input_data": input_data},
)

In [ ]:
ml_client.jobs.stream(name=endp_job.name)

### Analyze Outputs ###

In [ ]:
# Download output data
local_download_path = "../data/oj_sim_sales/outputs/"
ml_client.jobs.download(name=endp_job.name, download_path=local_download_path, output_name="pipeline_output")

In [ ]:
# Read output data as pandas df

glob_path = os.path.join(local_download_path +"named-outputs/pipeline_output/*.csv")
output_files = glob.glob(glob_path)
output_df = pd.concat((pd.read_csv(f) for f in output_files))

display(len(output_df))
display(output_df.head())

### Register Output as a AML Data Asset

In [ ]:
# Register predictions as a data asset

predictions_csv_path = "../data/oj_sim_sales/outputs/consolidated_outputs.csv"

output_df.to_csv(predictions_csv_path, index=False)

# set the version number of the data asset to the current UTC time
v = time.strftime("%Y.%m.%d.%H%M%S", time.gmtime())
local_path = "../data/oj_sim_sales/"

inference_output = Data(
    name="oj-sim-sales-predictions",
    version=v,
    description="Lab 02 output file",
    path=predictions_csv_path,
    type=AssetTypes.URI_FILE,
)

ml_client.data.create_or_update(inference_output)